In [ ]:
!pip install pandas-profiling==3.1.0

In [33]:
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.cluster import KMeans
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [53]:
data_set = pd.read_csv('dataset_customer_churn.csv', sep='^')
data_set = data_set.dropna()

In [ ]:
profile = ProfileReport(data_set)
profile

In [54]:
data_set = data_set.drop(['CD_ASSOCIADO','CODIGO_BENEFICIARIO','REALIZOU_EXODONTIA_COBERTA','REALIZOU_ENDODONTIA_COBERTA', 'A006_REGISTRO_ANS','A006_NM_PLANO','CD_USUARIO','CLIENTE','FORMA_PGTO_MENSALIDADE','QTDE_ATO_N_COBERTO_EXECUTADO','QTDE_ATENDIMENTOS'], axis=1)

In [66]:
data_set

,NUM_BENEFICIARIOS_FAMILIA,IDADE,SEXO,ESTADO_CIVIL,QTDE_DIAS_ATIVO,QTDE_ATO_COBERTO_EXECUTADO,REALIZOU_PROCEDIMEN_ALTO_CUSTO,DIAS_ATE_REALIZAR_ALTO_CUSTO,CODIGO_FORMA_PGTO_MENSALIDADE,PLANO
0,1.0,40.0,F,solteiro,277.0,8.0,NAO,NaoRealizou,D,408875991
1,2.0,56.0,F,casado,1725.0,10.0,SIM,91-200,D,408875991
2,2.0,66.0,F,casado,2006.0,73.0,SIM,31-60,D,408875991
3,3.0,62.0,M,casado,529.0,1.0,NAO,NaoRealizou,D,422427992
4,1.0,47.0,M,casado,176.0,0.0,NAO,NaoRealizou,B,408875991
...,...,...,...,...,...,...,...,...,...,...
386387,2.0,26.0,F,casado,55.0,11.0,NAO,NaoRealizou,T,481485181
386388,1.0,42.0,M,divorciado,55.0,1.0,NAO,NaoRealizou,T,481485181
386389,2.0,32.0,M,casado,55.0,7.0,NAO,NaoRealizou,T,481485181
386390,1.0,37.0,F,solteiro,833.0,5.0,NAO,NaoRealizou,B,408875991


In [55]:
y = data_set['SITUACAO']
data_set = data_set.drop(['SITUACAO'], axis=1)

dici_trad = {
    'DESATIVADO' : 0,
    'ATIVO':1
}

y =  y.replace(dici_trad)
y = pd.Series(y)

In [56]:
dummies = pd.get_dummies(data_set[['SEXO','ESTADO_CIVIL','REALIZOU_PROCEDIMEN_ALTO_CUSTO','DIAS_ATE_REALIZAR_ALTO_CUSTO','PLANO','CODIGO_FORMA_PGTO_MENSALIDADE']])

In [57]:
num = data_set.drop(['SEXO','ESTADO_CIVIL','REALIZOU_PROCEDIMEN_ALTO_CUSTO','DIAS_ATE_REALIZAR_ALTO_CUSTO','PLANO','CODIGO_FORMA_PGTO_MENSALIDADE'], axis=1)

In [58]:
x = pd.concat([dummies, num], axis=1)

VIEW FUNCTION

In [59]:
def view_score(y_teste, p) :
  baseline = np.ones(p.shape)

  ac = accuracy_score(y_teste,p)
  pc = precision_score(y_teste,p)
  f1 = f1_score(y_teste,p)

  print("Pelo Accuracy: ",ac * 100,"%")
  print("Pelo Precision: ",pc * 100,"%")
  print("Pelo F1: ",f1 * 100,"%")
  print()
  print("BASELINE")

  ac_b = accuracy_score(y_teste,baseline)
  pc_b = precision_score(y_teste,baseline)
  f1_b = f1_score(y_teste,baseline)

  print("Pelo Accuracy: ",ac_b * 100,"%")
  print("Pelo Precision: ",pc_b * 100,"%")
  print("Pelo F1: ",f1_b * 100,"%")

TESTE E TREINO

In [60]:
x = x
y = y

x_treino,x_teste,y_treino,y_teste = train_test_split(x, y, test_size=0.3, stratify=y)

RANDOM FOREST

In [61]:
rf = RandomForestClassifier()
rf.fit(x_treino, y_treino)

p = rf.predict(x_teste)

view_score(y_teste, p)

Pelo Accuracy:  89.3775459899847 %
Pelo Precision:  87.96358858858859 %
Pelo F1:  88.41495623302144 %

BASELINE
Pelo Accuracy:  45.61030244847478 %
Pelo Precision:  45.61030244847478 %
Pelo F1:  62.647081534102725 %


LOGISTIC REGRESSION

In [62]:
lr = LogisticRegression()

lr.fit(x_treino,y_treino)

p2 = lr.predict(x_teste)

view_score(y_teste,p2)

Pelo Accuracy:  81.32637969954075 %
Pelo Precision:  80.00077060897374 %
Pelo F1:  79.36699062526877 %

BASELINE
Pelo Accuracy:  45.61030244847478 %
Pelo Precision:  45.61030244847478 %
Pelo F1:  62.647081534102725 %


C:\Users\fossg\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


MULTILAYER

In [63]:
mlp = MLPClassifier()

mlp.fit(x_treino,y_treino)

p3 = mlp.predict(x_teste)

view_score(y_teste, p3)

Pelo Accuracy:  84.63368015014314 %
Pelo Precision:  85.61360627355128 %
Pelo F1:  82.55246437725252 %

BASELINE
Pelo Accuracy:  45.61030244847478 %
Pelo Precision:  45.61030244847478 %
Pelo F1:  62.647081534102725 %


CLUSTERING

In [68]:
data_final = pd.concat([x,y], axis=1)

values = Normalizer().fit_transform(data_final.values)

km = KMeans(n_clusters=5,n_init=10,max_iter=300)
km_pred = km.fit_predict(values)

clus = data_final

clus['CLUSTER'] = km.labels_
descp = clus.groupby('CLUSTER')['QTDE_DIAS_ATIVO','QTDE_ATO_COBERTO_EXECUTADO']
descp = pd.DataFrame(descp.mean())

print(descp)

         QTDE_DIAS_ATIVO  QTDE_ATO_COBERTO_EXECUTADO
CLUSTER                                             
0              97.297954                    2.268237
1            1083.989597                    7.574297
2              10.970534                    0.216186
3              41.820820                    0.957039
4             225.981770                    4.508428


C:\Users\fossg\AppData\Local\Temp\ipykernel_38028\280585691.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  descp = clus.groupby('CLUSTER')['QTDE_DIAS_ATIVO','QTDE_ATO_COBERTO_EXECUTADO']
